In [4]:
txt = """
1suit 45 %

2samesuit 6 %

1rank - 4.2% each in deck
    1specific 4.2 %

2rank - 0.37% each in deck
    2specific 0.1 %
    
gambler's ruin'

if p=0.5 => $p = n1/(n1+n2)$

"""

In [20]:
from card_games import TexasHoldem

In [26]:
x = TexasHoldem(num_players=4)
def f(n=100):
    for i in range(n):
        x.shuffle()
        x.winner()
        
%timeit f(100000)

1 loop, best of 3: 47.2 s per loop


In [32]:
x = TexasHoldem(num_players=5)
for i in range(10):
    x.shuffle(2,3)
    print(x.players(), x.table())

[['5♣', 'A♠'], ['T♥', 'K♥'], ['J♦', 'A♣'], ['2♣', '4♣'], ['9♥', '5♦']] ['2♠', '6♦', '6♠', '8♠', '5♠']
[['5♣', 'A♠'], ['T♥', 'K♥'], ['6♦', 'Q♠'], ['K♦', '5♦'], ['T♠', 'J♦']] ['3♦', '4♥', '6♠', '8♠', '5♠']
[['5♣', 'A♠'], ['T♥', 'K♥'], ['A♥', '5♥'], ['9♥', '4♦'], ['3♥', '2♦']] ['2♥', '2♣', '6♠', '8♠', '5♠']
[['5♣', 'A♠'], ['T♥', 'K♥'], ['2♥', 'T♣'], ['Q♣', '3♠'], ['2♣', 'A♣']] ['J♣', 'K♣', '6♠', '8♠', '5♠']
[['5♣', 'A♠'], ['T♥', 'K♥'], ['A♣', '6♦'], ['8♣', '5♥'], ['K♠', 'K♦']] ['A♥', '7♦', '6♠', '8♠', '5♠']
[['5♣', 'A♠'], ['T♥', 'K♥'], ['3♦', '9♥'], ['Q♠', '2♦'], ['8♦', '7♠']] ['3♥', '4♦', '6♠', '8♠', '5♠']
[['5♣', 'A♠'], ['T♥', 'K♥'], ['2♥', '3♣'], ['4♥', 'T♣'], ['4♣', '4♦']] ['2♠', '4♠', '6♠', '8♠', '5♠']
[['5♣', 'A♠'], ['T♥', 'K♥'], ['J♠', '8♣'], ['9♥', 'J♣'], ['5♦', '6♦']] ['J♦', '9♦', '6♠', '8♠', '5♠']
[['5♣', 'A♠'], ['T♥', 'K♥'], ['2♥', '2♣'], ['9♥', 'T♦'], ['Q♠', '4♥']] ['8♥', '2♠', '6♠', '8♠', '5♠']
[['5♣', 'A♠'], ['T♥', 'K♥'], ['3♠', '2♣'], ['7♣', '9♥'], ['Q♠', '3♥']] ['9♦', '3♣'

In [30]:
x = TexasHoldem(num_players=4)
for i in range(100):
    x.shuffle()
    w = x.winner()
    if w[0][0][0]>2:
        print(w)
        for h in  x.evaluate_players().items():
            print(h)
        print()

(((3, 'two pairs'), (14, 14, 3, 3)), [1])
(0, ((2, 'pair'), (3, 3), (3, 2)))
(1, ((3, 'two pairs'), (14, 14, 3, 3), (4, 2, 4, 2)))
(2, ((2, 'pair'), (13, 13), (4, 1)))
(3, ((2, 'pair'), (8, 8), (4, 1)))

(((3, 'two pairs'), (13, 13, 11, 11)), [0])
(0, ((3, 'two pairs'), (13, 13, 11, 11), (4, 2, 4, 3)))
(1, ((2, 'pair'), (4, 4), (3, 1)))
(2, ((1, 'highest rank'), (13,), (2,)))
(3, ((2, 'pair'), (3, 3), (3, 1)))

(((3, 'two pairs'), (12, 12, 3, 3)), [0])
(0, ((3, 'two pairs'), (12, 12, 3, 3), (3, 1, 3, 1)))
(1, ((2, 'pair'), (3, 3), (3, 1)))
(2, ((2, 'pair'), (3, 3), (3, 1)))
(3, ((2, 'pair'), (3, 3), (3, 1)))

(((4, 'three'), (7, 7, 7)), [3])
(0, ((2, 'pair'), (7, 7), (3, 2)))
(1, ((3, 'two pairs'), (7, 7, 6, 6), (3, 2, 4, 2)))
(2, ((3, 'two pairs'), (7, 7, 4, 4), (3, 2, 4, 2)))
(3, ((4, 'three'), (7, 7, 7), (3, 2, 1)))

(((5, 'straight'), (14, 13, 12, 11, 10)), [3])
(0, ((2, 'pair'), (13, 13), (2, 1)))
(1, ((1, 'highest rank'), (14,), (3,)))
(2, ((2, 'pair'), (3, 3), (4, 2)))
(3, ((5, 